In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import helper.flowtracing as ft
import helper.entsoe_wrapper as entsoe_wrapper
from entsoe import EntsoePandasClient
import helper.capacity as cap
from tqdm import tqdm

### Setup the Tracing

- 27 European Countries with Generation, Load, Import and Export
- Entsoe Client is used to load the data for the Generation and Load

In [2]:
# Define the start and end dates in UTC
start_utc = pd.Timestamp('2022-01-01', tz='UTC')
end_utc = pd.Timestamp('2023-01-01', tz='UTC')

# Codes for the Countries in the List
countryCodes=["AT","PT","ES","FR","IT","GR","ME","BG","RO","RS","HU","SK","SI","CZ","BE","NL","EE","LV","LT","FI","NO","SE","DK","PL","DE","IE"]

In [7]:
API_KEY="96ebcf8b-a543-4309-b167-322d5e0d5684"
client=EntsoePandasClient(api_key=API_KEY)

In [ ]:
entsoe_wrapper.get_import_data("PT",start_utc,end_utc).index

In [4]:
#   Load data
gen_dict,gen_types=entsoe.get_generation_dict(countryCodes,start_utc,end_utc,imputation_type="no")
import_dict=entsoe.get_import_dict(countryCodes,start_utc,end_utc,imputation_type="no") 
export_dict=entsoe.get_export_dict(countryCodes,start_utc,end_utc,imputation_type="no")

AT
PT


TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'

### Calculation

In [5]:
gen_types

{'Biomass',
 'Biomass_Actual Aggregated',
 'Biomass_Actual Consumption',
 'Fossil Brown coal/Lignite',
 'Fossil Brown coal/Lignite_Actual Aggregated',
 'Fossil Coal-derived gas',
 'Fossil Coal-derived gas_Actual Aggregated',
 'Fossil Gas',
 'Fossil Gas_Actual Aggregated',
 'Fossil Gas_Actual Consumption',
 'Fossil Hard coal',
 'Fossil Hard coal_Actual Aggregated',
 'Fossil Hard coal_Actual Consumption',
 'Fossil Oil',
 'Fossil Oil shale',
 'Fossil Oil shale_Actual Aggregated',
 'Fossil Oil_Actual Aggregated',
 'Fossil Oil_Actual Consumption',
 'Fossil Peat',
 'Fossil Peat_Actual Aggregated',
 'Fossil Peat_Actual Consumption',
 'Geothermal_Actual Aggregated',
 'Geothermal_Actual Consumption',
 'Hydro Pumped Storage',
 'Hydro Pumped Storage_Actual Aggregated',
 'Hydro Pumped Storage_Actual Consumption',
 'Hydro Run-of-river and poundage',
 'Hydro Run-of-river and poundage_Actual Aggregated',
 'Hydro Run-of-river and poundage_Actual Consumption',
 'Hydro Water Reservoir',
 'Hydro Water Re

In [5]:


def get_A_stacked(load_dict, export_dict, import_dict, countryCodes, timesteps):
    """
    Build the A matrix for all timesteps using numpy's vectorized operations.

    Parameters:
    - load_dict: Dictionary containing load data for each country.
    - export_dict: Dictionary containing export data for each country.
    - import_dict: Dictionary containing import data for each country.
    - countryCodes: List of country codes.
    - timesteps: Number of timesteps.

    Returns:
    - A: numpy array of shape (timesteps, n, n)
    """
    n = len(countryCodes)
    A = np.zeros((timesteps, n, n))

    # Fill the diagonal elements with load data and export data
    for i, country_code in enumerate(countryCodes):
        load_values = load_dict[country_code].values[:timesteps]
        A[:, i, i] = load_values.flatten()
        for key in export_dict[country_code].columns:
            export_values = export_dict[country_code][key].values[:timesteps]
            A[:, i, i] += export_values.flatten()

    # Fill the off-diagonal elements with import data
    for i, country_code in enumerate(countryCodes):
        for j, country_code2 in enumerate(countryCodes):
            if country_code != country_code2:
                try:
                    import_values = import_dict[country_code][country_code2].values[:timesteps]
                    A[:, i, j] = -import_values.flatten()
                except KeyError:
                    A[:, i, j] = 0

    return A

def get_b_stacked(gen_type, gen_dict, countryCodes, num_timesteps):
    b_stack = np.zeros((num_timesteps, len(countryCodes)))
    for country_code in countryCodes:
            b_stack[:,countryCodes.index(country_code)] = gen_dict[country_code][gen_type][:num_timesteps]
    return b_stack


In [6]:
from concurrent.futures import ThreadPoolExecutor

def solve_linear_system(A, b):
    """
    Solve the linear system Ax = b for a single timestep.
    """
    try:
        return np.linalg.solve(A, b)
    except np.linalg.LinAlgError:
        return np.full(b.shape, np.nan)  # Return NaNs if the system is singular

def solve_linear_systems_parallel(A, b):
    """
    Solve the linear systems Ax = b in parallel for all timesteps.

    Parameters:
    - A: numpy array of shape (timesteps, n, n)
    - b: numpy array of shape (timesteps, n)

    Returns:
    - x: numpy array of shape (timesteps, n) containing the solutions
    """
    timesteps = A.shape[0]
    n = A.shape[1]
    
    # Initialize the result array
    x = np.zeros((timesteps, n))
    
    # Use ThreadPoolExecutor to parallelize the computation
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(solve_linear_system, A, b))
    
    # Collect the results
    for i, result in enumerate(results):
        x[i] = result
    
    return x

In [7]:
gen_type='Wind Onshore'
num_timesteps=1

A_stack = get_A_stacked(gen_dict, export_dict, import_dict, countryCodes, num_timesteps) 
x=np.zeros((len(gen_types),num_timesteps,len(countryCodes)))
for i,gen_type in enumerate(gen_types):
    b_stack = get_b_stacked(gen_type, gen_dict, countryCodes, num_timesteps)
    x[i,:,:] = solve_linear_systems_parallel(A_stack, b_stack)

In [82]:
# Initialize an empty set for generation types
gen_types = set()
storage_dict = {}
# Load generation data for each country and update the set of generation types
for country in countryCodes:
    gen_df= utils.load_generation_data(country)
    gen_df= gen_df.resample('h').mean()
    if "Hydro Pumped Storage - Actual Consumption [MW]" in gen_df.columns:
        storage_dict[country] = gen_df["Hydro Pumped Storage - Actual Consumption [MW]"].fillna(0)
        gen_df.drop(columns=["Hydro Pumped Storage - Actual Consumption [MW]"], inplace=True)
    else:
        storage_dict[country] = pd.Series(np.zeros(len(gen_df)), index=gen_df.index)
    gen_dict[country] = gen_df
    
for gen_data in gen_dict.values():
    gen_types.update(gen_data.columns)
# Add all columns that are missing in each country's dataframe
for country_code in countryCodes:
    for gen_type in gen_types:
        if gen_type not in gen_dict[country_code].columns:
            gen_dict[country_code][gen_type] = 0


In [85]:
netimp_dict={}
for country_code in countryCodes:
    netimp_dict[country_code]=import_dict[country_code]-export_dict[country_code]

In [86]:
countrycodes_new = [country for country in countryCodes if gen_dict[country].iloc[0].isna().sum() < 1 and netimp_dict[country].iloc[0].isna().sum() < 1 ]

In [87]:
# Scale the generation based on imports from other countries
def scale_generation(gen_dict, import_dict, countries):
    scaled_gen_dict ={}
    # Calculate the total import from countries not included in the topology
    for country_code in countries:
            # total import from countries not included in the topology
            sum=0
            for country in import_dict[country_code].keys():
                  if country not in countries:
                        sum+=import_dict[country_code][country].iloc[0]          
            scaled_gen_dict[country_code] = gen_dict[country_code].iloc[0] * (1 + sum / gen_dict[country_code].iloc[0].sum())
    return scaled_gen_dict

In [88]:
scaled_gen_dict=scale_generation(gen_dict, import_dict, countrycodes_new)

In [89]:
# Setup the right hand side
# Numpy Vector with generation plus storage discharge
def get_b(gen_type, countryCodes):
    b = np.zeros(len(countryCodes))
    
    for i, country_code in enumerate(countryCodes):
        b[i] = scaled_gen_dict[country_code][gen_type]
    return b


In [90]:
def get_A(countrycodes_new):
    A=np.zeros((len(countrycodes_new),len(countrycodes_new)))
    for i,country_code in enumerate(countrycodes_new):
        for s,country_code2 in enumerate(countrycodes_new):
            if i==s:
                A[i,s]=(gen_dict[country_code].sum(axis=1).iloc[0])
                for key in netimp_dict[country_code].columns:
                        if key in countrycodes_new:
                            A[i,s] += netimp_dict[country_code][key].iloc[0]
                        else:
                            A[i,s] +=import_dict[country_code][key].iloc[0]
                            #A[i,s] -=export_dict[country_code][key].iloc[0] 
            else:
                if i<s:
                    try:
                        A[i,s]=-netimp_dict[country_code][country_code2].iloc[0]
                    except:
                        A[i,s]=0
                else:
                    try:
                        A[i,s]=-netimp_dict[country_code][country_code2].iloc[0]
                    except:
                        A[i,s]=0
    return A

In [91]:

q=dict()
A=get_A(countrycodes_new)

# Solve the linear equiation system for one technology
sum=0
for gen_type in gen_types:
    b=get_b(gen_type,countrycodes_new)
    sum+=b
    q[gen_type]=np.linalg.solve(A,b)


In [92]:
sum=0
for gen_type in gen_types:
    sum+=q[gen_type]